In [ ]:

import os, sys, numpy as np, pandas as pd, torch
sys.path.append("..")
from src.config import ModelParams, TrainConfig
from src.deqn import PolicyNetwork, Trainer
from src.steady_states import solve_flexprice_sss, export_rbar_tensor, solve_taylor_sss
from src.io_utils import ensure_dir, save_torch, save_csv, save_json, pack_config, save_selected_run


In [ ]:
policy = "taylor"
params = ModelParams(device="cpu", dtype=torch.float32)

# --- reproducible run directory (unique; safe for parallel runs) ---
from src.io_utils import make_run_dir
run_dir = make_run_dir("../artifacts", policy=policy, tag="FAST_PROJECT0", seed=123)

# Fast training plan (time-saving; keeps equations unchanged):
#   Stage A: gh_n=2 (quick stabilization)
#   Stage B: gh_n=3 (refine; no gh_n=5)

cfgA = TrainConfig(
    seed=123,
    steps=6000,
    batch_size=128,
    lr=2e-5,
    grad_clip=1.0,
    gh_n_train=2,
    replay_buffer_size=120_000,
    replay_refresh_every=80,
    replay_refresh_steps=60,
    replay_init_fill_steps=200,
    val_size=4096,
    val_every=400,
    reduce_lr_on_plateau=False,
    early_stopping=False,
    artifacts_root="../artifacts",
    run_dir=run_dir,
    save_best=True,
    enable_timers=True,
    log_every=100,
    profile=False,
    cpu_num_threads=4,
    cpu_num_interop_threads=1,
    matmul_precision="high",
)

cfgB = TrainConfig(
    seed=123,
    steps=4000,
    batch_size=96,
    lr=1e-5,
    grad_clip=1.0,
    gh_n_train=3,
    replay_buffer_size=120_000,
    replay_refresh_every=120,
    replay_refresh_steps=50,
    replay_init_fill_steps=0,
    val_size=4096,
    val_every=400,
    reduce_lr_on_plateau=False,
    early_stopping=False,
    artifacts_root="../artifacts",
    run_dir=run_dir,
    save_best=True,
    enable_timers=True,
    log_every=100,
    profile=False,
    cpu_num_threads=4,
    cpu_num_interop_threads=1,
    matmul_precision="high",
)


In [ ]:

flex = solve_flexprice_sss(params)
rbar = export_rbar_tensor(params, flex)


taylor_sss = solve_taylor_sss(params, flex)
print('Theoretical Taylor SSS:', taylor_sss.by_regime)


In [ ]:
def _quality_report(trainer, tag="", B_train=2048, B_val=4096, topk=5, burn_steps=200):
    import torch
    # Representative state draws (no dependency on internal replay buffer)
    css = commitment_sss if getattr(trainer, "policy", "") == "commitment" else None
    with torch.no_grad():
        Xtr = trainer.simulate_initial_state(B_train, commitment_sss=css)
        for _ in range(burn_steps):
            Xtr = trainer._step_state(Xtr)
        Xva = trainer.simulate_initial_state(B_val, commitment_sss=css)
        for _ in range(burn_steps):
            Xva = trainer._step_state(Xva)

    def _eval(X):
        if trainer.policy in ("discretion", "commitment"):
            with torch.enable_grad():
                R = trainer._residuals(X)
                L = float(((R**2).mean()).detach().cpu())
                rmse = float(torch.sqrt((R**2).mean()).detach().cpu())
                rmse_eq = torch.sqrt((R.detach() ** 2).mean(dim=0)).cpu()
        else:
            with torch.inference_mode():
                R = trainer._residuals(X)
                L = float(((R**2).mean()).detach().cpu())
                rmse = float(torch.sqrt((R**2).mean()).detach().cpu())
                rmse_eq = torch.sqrt((R.detach() ** 2).mean(dim=0)).cpu()
        return L, rmse, rmse_eq

    Ltr, rtr, _ = _eval(Xtr)
    Lva, rva, rmse_eq_va = _eval(Xva)

    k = min(int(topk), rmse_eq_va.numel())
    worst = torch.topk(rmse_eq_va, k=k).indices.tolist()
    worst_txt = ", ".join([f"{trainer.res_keys[j]}:{rmse_eq_va[j].item():.2e}" for j in worst])

    print(f"[qreport] {tag} | train_loss={Ltr:.3e} train_RMSE={rtr:.3e} | val_loss={Lva:.3e} val_RMSE={rva:.3e} | worst={worst_txt}")


In [ ]:
from src.deqn import simulate_paths
x0 = trainer.simulate_initial_state(256)
sim = simulate_paths(params, policy, trainer.net, T=3000, burn_in=600, x0=x0, compute_implied_i=False)
for s in [0,1]:
    mask = (sim['s'] == s)
    print(f'Regime {s}: pi_sim={sim["pi"][mask].mean():.6f} | pi_sss={taylor_sss.by_regime[s]["pi"]:.6f}')


In [ ]:
# --- Save artifacts (final + pointers) ---
# We save final weights/logs to run_dir and also keep a pointer in selected_runs.json

# prefer Stage B config if it exists, else Stage A
_cfg_final = globals().get("cfgB", globals().get("cfgA"))
save_run_metadata(run_dir, pack_config(params, _cfg_final, extra={"policy": policy}))

save_torch(os.path.join(run_dir, "weights_final.pt"), trainer.net.state_dict())
# best checkpoint is saved automatically during training as cfg.best_weights_name when val improves

df = pd.DataFrame({"step": np.arange(len(losses)), "loss": losses})
save_csv(os.path.join(run_dir, "train_log.csv"), df)

meta = pack_config(params, _cfg_final, extra={"policy": policy, "flex_sss": flex.by_regime})
save_json(os.path.join(run_dir, "meta.json"), meta)

pointer_path = save_selected_run(_cfg_final.artifacts_root, policy, run_dir)
print("Saved to", run_dir)
print("Selected run pointer:", pointer_path)


In [ ]:
# Final cleanup (helps when running multiple models sequentially)
import gc
try:
    del trainer
except NameError:
    pass
try:
    del net
except NameError:
    pass
_ = gc.collect()
print('final gc.collect() done')
